In [1]:
# API key ID: otlwwfxero72fkms03hi49x2
# API key secret: 5qbnpzqphl1y8bz6eggxraxlzlnbnhso49bro4925r1rpxj6sn
# app token: 8y23ArYxg4pcvOMgWzMFSC0nB
# secret app token: pZgwEL4lKm2r1s868Saz3EJ1ebpTKy-FzglU

# crime data docs: https://dev.socrata.com/foundry/data.cityofchicago.org/9hwr-2zxp

# make sure to install these packages before running:
!pip install pandas
!pip install sodapy
!pip install category_encoders

In [2]:
import pandas as pd
import sodapy
from sodapy import Socrata
import statistics
pd.set_option('display.max_columns', 500)

In [3]:
# crime_url = "https://data.cityofchicago.org/resource/9hwr-2zxp.json"


app_token = "8y23ArYxg4pcvOMgWzMFSC0nB"

# Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                 app_token, 
                 username="ch35@illinois.edu",
                 password="68M8rz3vF!StBjD")


# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("9hwr-2zxp", limit = 250000)

# Convert to pandas DataFrame
crime_df = pd.DataFrame.from_records(results)

print(crime_df.shape)

(201432, 22)


# Logistic Regression

In [4]:
crime_df.head()

,date,location,district,block,y_coordinate,latitude,description,location_description,updated_on,community_area,iucr,x_coordinate,ward,case_number,year,domestic,fbi_code,longitude,beat,primary_type,arrest,id
0,2022-11-09T23:45:00.000,"{'latitude': '41.853718318', 'human_address': ...",012,021XX S WOOD ST,1889981,41.853718318,AGGRAVATED - HANDGUN,APARTMENT,2022-11-16T15:51:52.000,31,041A,1164698,25,JF468923,2022,False,04B,-87.670980997,1234,BATTERY,False,12888055
1,2022-11-09T23:45:00.000,"{'latitude': '41.965248711', 'human_address': ...",017,046XX N FRANCISCO AVE,1930561,41.965248711,OVER $500,STREET,2022-11-16T15:51:52.000,14,0810,1156210,33,JF469910,2022,False,06,-87.701037211,1713,THEFT,False,12888758
2,2022-11-09T23:30:00.000,"{'latitude': '41.772532392', 'human_address': ...",003,067XX S JEFFERY BLVD,1860610,41.772532392,AUTOMOBILE,STREET,2022-11-16T15:51:52.000,43,0910,1190707,5,JF469056,2022,False,07,-87.57646936,0331,MOTOR VEHICLE THEFT,False,12888163
3,2022-11-09T23:30:00.000,"{'latitude': '41.885325435', 'human_address': ...",001,001XX N STATE ST,1901592,41.885325435,SIMPLE,SIDEWALK,2022-11-16T15:51:52.000,32,0460,1176332,42,JF468948,2022,False,08B,-87.627930066,0111,BATTERY,False,12888071
4,2022-11-09T23:26:00.000,"{'latitude': '41.93939819', 'human_address': '...",017,027XX W BELMONT AVE,1921150,41.93939819,DOMESTIC BATTERY SIMPLE,APARTMENT,2022-11-16T15:51:52.000,21,0486,1157497,33,JF468940,2022,True,08B,-87.696562292,1733,BATTERY,True,12888129


In [5]:
#creating variable for month
month = [i[0:7] for i in crime_df.date]

In [6]:
#adding to data frame
crime_df["month"] = month

In [7]:
#dropping irrelevant variables
crime_df_clean = crime_df.drop(["date", "location", "block", "updated_on", "case_number", "year", "id"], axis = 1)
crime_df_clean.head()

,district,y_coordinate,latitude,description,location_description,community_area,iucr,x_coordinate,ward,domestic,fbi_code,longitude,beat,primary_type,arrest,month
0,012,1889981,41.853718318,AGGRAVATED - HANDGUN,APARTMENT,31,041A,1164698,25,False,04B,-87.670980997,1234,BATTERY,False,2022-11
1,017,1930561,41.965248711,OVER $500,STREET,14,0810,1156210,33,False,06,-87.701037211,1713,THEFT,False,2022-11
2,003,1860610,41.772532392,AUTOMOBILE,STREET,43,0910,1190707,5,False,07,-87.57646936,0331,MOTOR VEHICLE THEFT,False,2022-11
3,001,1901592,41.885325435,SIMPLE,SIDEWALK,32,0460,1176332,42,False,08B,-87.627930066,0111,BATTERY,False,2022-11
4,017,1921150,41.93939819,DOMESTIC BATTERY SIMPLE,APARTMENT,21,0486,1157497,33,True,08B,-87.696562292,1733,BATTERY,True,2022-11


## Removing Categorical Variables with High Cardinality (50+ Levels)

In [8]:
crime_df_clean.dtypes

district                object
y_coordinate            object
latitude                object
description             object
location_description    object
community_area          object
iucr                    object
x_coordinate            object
ward                    object
domestic                  bool
fbi_code                object
longitude               object
beat                    object
primary_type            object
arrest                    bool
month                   object
dtype: object

In [9]:
#converting variables to numeric
num_vars = ["y_coordinate", "x_coordinate", "latitude", "longitude"]
for i in num_vars:
    crime_df_clean[i] = crime_df_clean[i].astype("float")
crime_df_clean.dtypes

district                 object
y_coordinate            float64
latitude                float64
description              object
location_description     object
community_area           object
iucr                     object
x_coordinate            float64
ward                     object
domestic                   bool
fbi_code                 object
longitude               float64
beat                     object
primary_type             object
arrest                     bool
month                    object
dtype: object

In [10]:
#converting boolean variables to binary categorical variables
crime_df_clean["domestic"] = crime_df_clean["domestic"].astype(int)
crime_df_clean["arrest"] = crime_df_clean["arrest"].astype(int)
crime_df_clean.dtypes

district                 object
y_coordinate            float64
latitude                float64
description              object
location_description     object
community_area           object
iucr                     object
x_coordinate            float64
ward                     object
domestic                  int64
fbi_code                 object
longitude               float64
beat                     object
primary_type             object
arrest                    int64
month                    object
dtype: object

In [11]:
crime_df_clean["arrest"].dtypes == "O"

False

In [12]:
#storing list of cateogorical variables with high cardinality
high_card = [var for var in crime_df_clean.columns if len(crime_df_clean[var].value_counts()) >= 50 and crime_df_clean[var].dtypes == "O"]
high_card

['description',
 'location_description',
 'community_area',
 'iucr',
 'ward',
 'beat']

In [13]:
#dropping from set
crime_df_new = crime_df_clean.drop(high_card, axis = 1)
crime_df_new.head()

,district,y_coordinate,latitude,x_coordinate,domestic,fbi_code,longitude,primary_type,arrest,month
0,012,1889981.0,41.853718,1164698.0,0,04B,-87.670981,BATTERY,0,2022-11
1,017,1930561.0,41.965249,1156210.0,0,06,-87.701037,THEFT,0,2022-11
2,003,1860610.0,41.772532,1190707.0,0,07,-87.576469,MOTOR VEHICLE THEFT,0,2022-11
3,001,1901592.0,41.885325,1176332.0,0,08B,-87.627930,BATTERY,0,2022-11
4,017,1921150.0,41.939398,1157497.0,1,08B,-87.696562,BATTERY,1,2022-11


## Binning Categorical Variables
#### Goal is to group levels together if they share a similar target rate to establish bin credibility (at least 1% of observations in each bin)

In [14]:
cat_vars = [var for var in crime_df_new.columns if crime_df_new[var].dtypes == "O"]
cat_vars

['district', 'fbi_code', 'primary_type', 'month']

In [15]:
#district
pd.DataFrame({"District": crime_df_new["district"].value_counts().index, "Target Rate": [round(statistics.mean(crime_df_new["arrest"].loc[crime_df_new["district"] == i]), 3) for i in crime_df_new["district"].value_counts().index], "Proportion": [round(crime_df_new.loc[crime_df_new["district"] == i].shape[0] / crime_df_new.shape[0], 3) for i in crime_df_new["district"].value_counts().index]}).sort_values(by = "Target Rate")
                                                                                                                                                                                                                               

,District,Target Rate,Proportion
22,16,0.000,0.000
7,019,0.072,0.051
2,012,0.082,0.059
10,002,0.086,0.049
21,020,0.087,0.021
1,008,0.092,0.062
8,003,0.092,0.050
3,004,0.093,0.059
17,024,0.098,0.036
19,022,0.099,0.032


##### Decision: 16 + 019, 011 + 031, all others can remain on their own

In [16]:
#FBI Code
pd.DataFrame({"FBI Code": crime_df_new["fbi_code"].value_counts().index, "Target Rate": [round(statistics.mean(crime_df_new["arrest"].loc[crime_df_new["fbi_code"] == i]), 3) for i in crime_df_new["fbi_code"].value_counts().index], "Proportion": [round(crime_df_new.loc[crime_df_new["fbi_code"] == i].shape[0] / crime_df_new.shape[0], 3) for i in crime_df_new["fbi_code"].value_counts().index]}).sort_values(by = "Target Rate")
                                                                                                                                                                                                                               

,FBI Code,Target Rate,Proportion
23,12,0.000,0.000
13,10,0.007,0.009
6,11,0.013,0.053
14,02,0.021,0.007
3,07,0.024,0.083
2,14,0.032,0.115
0,06,0.035,0.232
10,05,0.038,0.032
8,03,0.048,0.037
15,17,0.053,0.007


##### Decision: Variable incredibly similar to the variable below, we'll drop this one

In [17]:
crime_df_new = crime_df_new.drop(["fbi_code"], axis = 1)
crime_df_new.head()

,district,y_coordinate,latitude,x_coordinate,domestic,longitude,primary_type,arrest,month
0,012,1889981.0,41.853718,1164698.0,0,-87.670981,BATTERY,0,2022-11
1,017,1930561.0,41.965249,1156210.0,0,-87.701037,THEFT,0,2022-11
2,003,1860610.0,41.772532,1190707.0,0,-87.576469,MOTOR VEHICLE THEFT,0,2022-11
3,001,1901592.0,41.885325,1176332.0,0,-87.627930,BATTERY,0,2022-11
4,017,1921150.0,41.939398,1157497.0,1,-87.696562,BATTERY,1,2022-11


In [18]:
#Primary Type
pd.DataFrame({"Primary Type": crime_df_new["primary_type"].value_counts().index, "Target Rate": [round(statistics.mean(crime_df_new["arrest"].loc[crime_df_new["primary_type"] == i]), 3) for i in crime_df_new["primary_type"].value_counts().index], "Proportion": [round(crime_df_new.loc[crime_df_new["primary_type"] == i].shape[0] / crime_df_new.shape[0], 3) for i in crime_df_new["primary_type"].value_counts().index]}).sort_values(by = "Target Rate")
                                                                                                                                                                                                                               

,Primary Type,Target Rate,Proportion
26,HUMAN TRAFFICKING,0.000,0.000
23,INTIMIDATION,0.007,0.001
5,DECEPTIVE PRACTICE,0.015,0.063
13,CRIMINAL SEXUAL ASSAULT,0.020,0.007
4,MOTOR VEHICLE THEFT,0.024,0.083
2,CRIMINAL DAMAGE,0.032,0.115
0,THEFT,0.035,0.232
9,BURGLARY,0.038,0.032
18,STALKING,0.044,0.002
8,ROBBERY,0.048,0.037


##### Decision: Bottom 5, Next 5, Next 7, Next 2, Next 5, Last 7

In [19]:
#Month
pd.DataFrame({"Month": crime_df_new["month"].value_counts().index, "Target Rate": [round(statistics.mean(crime_df_new["arrest"].loc[crime_df_new["month"] == i]), 3) for i in crime_df_new["month"].value_counts().index], "Proportion": [round(crime_df_new.loc[crime_df_new["month"] == i].shape[0] / crime_df_new.shape[0], 3) for i in crime_df_new["month"].value_counts().index]}).sort_values(by = "Target Rate")
                                                                                                                                                                                                                               

,Month,Target Rate,Proportion
10,2022-11,0.081,0.031
0,2022-10,0.089,0.111
3,2022-09,0.101,0.109
1,2022-07,0.104,0.109
2,2022-08,0.105,0.109
8,2022-01,0.111,0.077
4,2022-06,0.118,0.102
6,2022-03,0.121,0.090
7,2022-04,0.130,0.088
5,2022-05,0.138,0.099


##### Decision: Bins good to remain on their own, may consider seasonal binning later

### Implementing Bins

In [20]:
#district
crime_df_new.loc[crime_df_new["district"].isin(["16", "019"]), "district"] = "16/019"
crime_df_new.loc[crime_df_new["district"].isin(["011", "031"]), "district"] = "011/031"

In [21]:
#checking new values
pd.DataFrame({"District": crime_df_new["district"].value_counts().index, "Target Rate": [round(statistics.mean(crime_df_new["arrest"].loc[crime_df_new["district"] == i]), 3) for i in crime_df_new["district"].value_counts().index], "Proportion": [round(crime_df_new.loc[crime_df_new["district"] == i].shape[0] / crime_df_new.shape[0], 3) for i in crime_df_new["district"].value_counts().index]}).sort_values(by = "Target Rate")
                                                                                                                                                                                                                               

,District,Target Rate,Proportion
7,16/019,0.071,0.051
2,012,0.082,0.059
10,002,0.086,0.049
21,020,0.087,0.021
1,008,0.092,0.062
8,003,0.092,0.050
3,004,0.093,0.059
17,024,0.098,0.036
19,022,0.099,0.032
0,006,0.104,0.062


In [22]:
#Primary Type
crime_df_new.loc[crime_df_new["primary_type"].isin(["HUMAN TRAFFICKING", "INTIMIDATION", "DECEPTIVE PRACTICE", "CRIMINAL SEXUAL ASSAULT", "MOTOR VEHICLE THEFT"]), "primary_type"] = "bin1"
crime_df_new.loc[crime_df_new["primary_type"].isin(["CRIMINAL DAMAGE", "THEFT", "BURGLARY", "STALKING", "ROBBERY"]), "primary_type"] = "bin2"
crime_df_new.loc[crime_df_new["primary_type"].isin(["OFFENSE INVOLVING CHILDREN", "SEX OFFENSE", "ARSON", "KIDNAPPING", "ASSAULT", "BATTERY", "OTHER OFFENSE"]), "primary_type"] = "bin3"
crime_df_new.loc[crime_df_new["primary_type"].isin(["HOMICIDE", "CRIMINAL TRESPASS"]), "primary_type"] = "bin4"
crime_df_new.loc[crime_df_new["primary_type"].isin(["OTHER NARCOTIC VIOLATION", "PUBLIC PEACE VIOLATION", "NON-CRIMINAL", "WEAPONS VIOLATION", "OBSCENITY"]), "primary_type"] = "bin5"
crime_df_new.loc[crime_df_new["primary_type"].isin(["PUBLIC INDECENCY", "GAMBLING", "INTERFERENCE WITH PUBLIC OFFICER", "NARCOTICS", "CONCEALED CARRY LICENSE VIOLATION", "LIQUOR LAW VIOLATION", "PROSTITUTION"]), "primary_type"] = "bin6"


In [23]:
#checking new values
pd.DataFrame({"Primary Type": crime_df_new["primary_type"].value_counts().index, "Target Rate": [round(statistics.mean(crime_df_new["arrest"].loc[crime_df_new["primary_type"] == i]), 3) for i in crime_df_new["primary_type"].value_counts().index], "Proportion": [round(crime_df_new.loc[crime_df_new["primary_type"] == i].shape[0] / crime_df_new.shape[0], 3) for i in crime_df_new["primary_type"].value_counts().index]}).sort_values(by = "Target Rate")
                                                                                                                                                                                                                               

,Primary Type,Target Rate,Proportion
2,bin1,0.020,0.153
0,bin2,0.036,0.418
1,bin3,0.124,0.345
5,bin4,0.289,0.021
3,bin5,0.613,0.042
4,bin6,0.950,0.022


# WoE (Weight of Evidence) Coding Categorical Variables
### Converts categoricals to numerics to allow their use in the model

In [85]:
#dropping na values
crime_df_new = crime_df_new.dropna()

In [86]:
#train / test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(crime_df_new.drop(["arrest"], axis = 1), crime_df_new["arrest"], shuffle=True)

In [87]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(150025, 8)
(50009, 8)
(150025,)
(50009,)


In [88]:
#making sure overall target rates are the same
print(statistics.mean(y_train.astype(int)))
print(statistics.mean(y_test.astype(int)))

0.11389435094150975
0.11293967085924533


In [89]:
#variables to be woe coded
cat_vars_new = [var for var in crime_df_new if crime_df_new[var].dtypes == "O" or var == "domestic"]
cat_vars_new

['district', 'domestic', 'primary_type', 'month']

In [90]:
import category_encoders as ce

In [91]:
#implementing woe coding
woe = ce.WOEEncoder(cols = cat_vars_new)
woe.fit(X_train, y_train)
train_woe = woe.transform(X_train, y_train)
test_woe = woe.transform(X_test)

In [92]:
train_woe.head()

,district,y_coordinate,latitude,x_coordinate,domestic,longitude,primary_type,month
151404,0.707980,1905022.0,41.895314,1148788.0,-0.02094,-87.728988,5.062243,0.168570
77729,0.066606,1895425.0,41.868356,1178412.0,-0.02094,-87.620480,0.096296,-0.104760
163166,-0.132281,1850883.0,41.746212,1174708.0,-0.02094,-87.635406,5.062243,0.094057
58246,0.039760,1905735.0,41.896774,1172755.0,-0.02094,-87.640943,-1.230139,-0.117608
195172,0.029829,1857560.0,41.764572,1172984.0,0.08956,-87.641526,0.096296,-0.002661


In [93]:
test_woe.head()

,district,y_coordinate,latitude,x_coordinate,domestic,longitude,primary_type,month
115564,0.029829,1861356.0,41.775145,1165793.0,-0.02094,-87.667776,-1.230139,0.230392
83511,-0.132281,1849896.0,41.743299,1183675.0,-0.02094,-87.602580,-1.230139,-0.104760
140031,-0.132281,1848381.0,41.739285,1177422.0,-0.02094,-87.625537,0.096296,0.168570
77590,-0.248562,1860389.0,41.772161,1180741.0,-0.02094,-87.613008,-1.230139,-0.104760
58514,-0.337163,1906972.0,41.900506,1156861.0,0.08956,-87.699285,0.096296,-0.117608


# Univariate Screening
#### Generating univariate models to see which are predictive on their own

In [97]:
variables = train_woe.columns
import statsmodels.api as sm
from sklearn import metrics

In [99]:
#getting AUC for each individual variable
AUC = []
count = 0
for i in variables:
    mod = sm.GLM(y_train, train_woe[i], family=sm.families.Binomial()).fit()
    pred = mod.predict(test_woe[i])
    fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
    AUC.append(metrics.auc(fpr, tpr))

In [101]:
#displaying AUCs alongside their predictor
pd.DataFrame({"Predictor": train_woe.columns, "AUC": AUC})

,Predictor,AUC
0,district,0.579459
1,y_coordinate,0.500329
2,latitude,0.500158
3,x_coordinate,0.536116
4,domestic,0.513048
5,longitude,0.463532
6,primary_type,0.812202
7,month,0.541236


#### All predictors above .5 (random guessing) will be kept in for consideration

In [103]:
train_woe = train_woe.drop(["longitude"], axis = 1)
test_woe = test_woe.drop(["longitude"], axis = 1)

# Time Series (TO BE DONE)

In [ ]:
#creating data frames
crime_agg = crime_df.groupby(by=month).size()
crime_sum = crime_df.groupby(by=month).sum()

In [ ]:
#number of crimes per month
crime_agg